In [1]:
import numpy as np
import pandas as pd
from sunpy.net import hek
from shapely import wkt
from sunpy.time import parse_time
import datetime

Download as csv lists of solar events 

In [2]:
client = hek.HEKClient()
start_time = pd.to_datetime('2011/03/01 00:00')
end_time = pd.to_datetime('2016/06/01 00:00')

In [2]:

fl_detective = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('FL'),
                                  hek.attrs.FRM.Name == 'Flare Detective - Trigger Module')
fl_detective = pd.DataFrame(fl_detective)
fl_detective.to_csv('raw_detective.csv', index = False)
       

In [5]:
fl_SSW = client.query(hek.attrs.Time(start_time.isoformat(),end_time.isoformat()),
                                  hek.attrs.EventType('FL'), hek.attrs.FL.GOESCls != None)
fl_SSW = pd.DataFrame(fl_SSW)
fl_SSW.to_csv('raw_SSW1.csv', index = False)

In [9]:
swpc = fl_SSW.ix[fl_SSW['search_frm_name']=='SWPC']
ssw = fl_SSW.ix[fl_SSW['search_frm_name']=='SSW Latest Events']
print ssw.shape[0]
print swpc.shape[0]
swpc_pos = swpc.ix[swpc['noposition']=='false']
print swpc['noposition'].values[0]
print swpc_pos.shape[0]

10556
11217
true
4301


In [9]:
client = hek.HEKClient()
start_time = pd.to_datetime('2010/06/01 00:00')
end_time = pd.to_datetime('2016/06/01 00:00')
ef_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('EF'))
ef = pd.DataFrame(ef_events)
ef.to_csv('raw_ef.csv', index = False)

In [3]:
ar_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('AR'), hek.attrs.FRM.Name == 'NOAA SWPC Observer')
ar = pd.DataFrame(ar_events)
ar.to_csv('raw_ar.csv', index = False)

NameError: name 'ef' is not defined

In [4]:
ar.to_csv('raw_ar.csv', index = False)

In [ ]:
cme_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('CE'))
cme = pd.DataFrame(cme_events)

In [10]:
sg_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('SG'))
sg = pd.DataFrame(sg_events)
sg.to_csv('raw_sg.csv', index = False)

In [ ]:
fe_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('FE'))
fe = pd.DataFrame(fe_events)

In [12]:
er_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('ER'))
er = pd.DataFrame(er_events)
er.to_csv('raw_er.csv', index = False)

In [ ]:
# fl_detective = client.query(hek.attrs.Time(start_time.isoformat(),
#                                                  end_time.isoformat()),
#                                   hek.attrs.EventType('FL'),
#                                   hek.attrs.FRM.Name == 'Flare Detective - Trigger Module')
# fl_SSW= client.query(hek.attrs.Time(start_time.isoformat(),
#                                                  end_time.isoformat()),
#                                   hek.attrs.EventType('FL'),
#                                   hek.attrs.FRM.Name == 'SSW Latest Events')

In [2]:
#may at some point add location of other flares in as well as peak flux
def determine_wavelengths(df, index, index2 = None):
    if index2 == None:
        if df['obs_channelid'].values[index] == '131':
            df['is_131'].values[index] = 1
            df['fl_peakflux_131'].values[index] = df['fl_peakflux'].values[index]
            
        if df['obs_channelid'].values[index] == '171':
            df['is_171'].values[index] = 1
            df['fl_peakflux_171'].values[index] = df['fl_peakflux'].values[index]
        
        if df['obs_channelid'].values[index] == '193':
            df['is_193'].values[index] = 1
            df['fl_peakflux_193'].values[index] = df['fl_peakflux'].values[index]
        
        if df['obs_channelid'].values[index] == '211':
            df['is_211'].values[index] = 1
            df['fl_peakflux_211'].values[index] = df['fl_peakflux'].values[index]
        
        if df['obs_channelid'].values[index] == '304':
            df['is_304'].values[index] = 1
            df['fl_peakflux_304'].values[index] = df['fl_peakflux'].values[index]
        
        if df['obs_channelid'].values[index] == '355':
            df['is_355'].values[index] = 1
            df['fl_peakflux_355'].values[index] = df['fl_peakflux'].values[index]
        
        if df['obs_channelid'].values[index] == '94':
            df['is_94'].values[index] = 1
            df['fl_peakflux_94'].values[index] = df['fl_peakflux'].values[index]
    else:
        if df['obs_channelid'].values[index2] == '131':
            df['is_131'].values[index] = 1
            df['fl_peakflux_131'].values[index] = df['fl_peakflux'].values[index2]
        
        if df['obs_channelid'].values[index2] == '171':
            df['is_171'].values[index] = 1
            df['fl_peakflux_171'].values[index] = df['fl_peakflux'].values[index2]
        
        if df['obs_channelid'].values[index2] == '193':
            df['is_193'].values[index] = 1
            df['fl_peakflux_193'].values[index] = df['fl_peakflux'].values[index2]
        
        if df['obs_channelid'].values[index2] == '211':
            df['is_211'].values[index] = 1
            df['fl_peakflux_211'].values[index] = df['fl_peakflux'].values[index2]
           
        if df['obs_channelid'].values[index2] == '304':
            df['is_304'].values[index] = 1
            df['fl_peakflux_304'].values[index] = df['fl_peakflux'].values[index2]
        
        if df['obs_channelid'].values[index2] == '355':
            df['is_355'].values[index] = 1
            df['fl_peakflux_355'].values[index] = df['fl_peakflux'].values[index2]
        
        if df['obs_channelid'].values[index2] == '94':
            df['is_94'].values[index] = 1
            df['fl_peakflux_94'].values[index] = df['fl_peakflux'].values[index2]

    return df

In [ ]:
def fl_duplicates_detective(fl_detective, delta_t_max, distance_max, search_index):
    delete_index = []
    num_del = 0
    num_duplicates = 0
    i = 0
    associated_fl = []
    detective_start = map(parse_time, fl_detective['event_starttime'])
    detective_position = map(lambda x: wkt.loads(x), fl_detective['hpc_coord'])
    delta_t_max = datetime.timedelta(minutes = delta_t_max)
    num_events = fl_detective.shape[0]
    
    #create columns in dataframe for recording multiple wavelength components of flares
    zero_list = [0 for n in range(num_events)]
    is_associated_fl = zero_list
    
    fl_detective.loc[:, 'is_131'] = zero_list
    fl_detective.loc[:, 'is_171'] = zero_list
    fl_detective.loc[:, 'is_193'] = zero_list
    fl_detective.loc[:, 'is_211'] = zero_list
    fl_detective.loc[:, 'is_304'] = zero_list
    fl_detective.loc[:, 'is_355'] = zero_list
    fl_detective.loc[:, 'is_355'] = zero_list
    fl_detective.loc[:, 'is_94'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_131'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_171'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_193'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_211'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_304'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_355'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_94'] = zero_list
    for elem in detective_start:
        fl_match = []
        determine_wavelengths(fl_detective, i)
        #test whether next entry is redundant
        if i < num_events-1:
            if fl_detective['obs_channelid'].values[i] == fl_detective['obs_channelid'].values[i+1]:
                if detective_start[i+1]-elem <= delta_t_max:
                    if detective_position[i].distance(detective_position[i+1]) <= distance_max:
                        delete_index.append(i+1)
                        fl_match.append(fl_detective['SOL_standard'].values[i+1])
                        num_del+=1
        #test whether there are entries in diff wavelengths for the same flare
        for j in range(1, search_index):
            i2 = i+j
            if i2 < (num_events-1):
                if fl_detective['obs_channelid'].values[i] != fl_detective['obs_channelid'].values[i2]:
                    if detective_start[i2]-elem <= delta_t_max:
                        if detective_position[i].distance(detective_position[i2]) <= distance_max:
                            delete_index.append(i2)
                            num_duplicates+=1
                            fl_match.append(str(fl_detective['SOL_standard'].values[i2]))
                            determine_wavelengths(fl_detective, i, i2)
        if fl_match == []: 
             fl_match = [0]
        associated_fl.append(fl_match)
        i+=1
    fl_detective.loc[:, 'associated_fl'] = associated_fl
    k = 0
    for elem in associated_fl:
        if elem!=[0]:
            is_associated_fl[k] = 1
        k+=1
    delete_index = set(delete_index)
    fl_detective.loc[:, 'is_associated_fl'] = is_associated_fl
    fl_detective = fl_detective.drop(delete_index) 
    
    print 'original number of events:  %d' % num_events
    print 'duplicated events deleted:  %d' % num_del
    print 'duplicated events merged:  %d' % (len(delete_index)-num_del)
    print 'new number of events:  %d' % (num_events-len(delete_index))

    return fl_detective     

In [6]:
def fl_duplicates_detective2(fl_detective, delta_t_max, distance_max, search_index):
    delete_index = []
    delete_index2 = []
    num_del = 0
    num_duplicates = 0
    i = 0
    associated_fl = []
    detective_start = map(parse_time, fl_detective['event_starttime'])
    detective_position = map(lambda x: wkt.loads(x), fl_detective['hpc_coord'])
    delta_t_max = datetime.timedelta(minutes = delta_t_max)
    num_events = fl_detective.shape[0]
    
    #create columns in dataframe for recording multiple wavelength components of flares
    zero_list = [0 for n in range(num_events)]
    is_associated_fl = zero_list
    idx = fl_detective['event_starttime'].index.tolist()
    
    fl_detective.loc[:, 'is_131'] = zero_list
    fl_detective.loc[:, 'is_171'] = zero_list
    fl_detective.loc[:, 'is_193'] = zero_list
    fl_detective.loc[:, 'is_211'] = zero_list
    fl_detective.loc[:, 'is_304'] = zero_list
    fl_detective.loc[:, 'is_335'] = zero_list
    fl_detective.loc[:, 'is_94'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_131'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_171'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_193'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_211'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_304'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_335'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_94'] = zero_list
    for elem in detective_start:
        fl_match = []
        fl_detective = determine_wavelengths(fl_detective, i)
        #test whether next entry is redundant
        if i < num_events-1:
            if fl_detective['obs_channelid'].values[i] == fl_detective['obs_channelid'].values[i+1]:
                if detective_start[i+1]-elem <= delta_t_max:
                    if detective_position[i].distance(detective_position[i+1]) <= distance_max:
                        delete_index2.append(idx[i+1])
                        fl_match.append(fl_detective['SOL_standard'].values[i+1])
                        #fl_detective = fl_detective.drop(idx[i+1])
                        
                        num_del+=1
        #test whether there are entries in diff wavelengths for the same flare
        for j in range(1, search_index):
            i2 = i+j
            if i2 < (num_events-1):
                if fl_detective['obs_channelid'].values[i] != fl_detective['obs_channelid'].values[i2]:
                    if detective_start[i2]-elem <= delta_t_max:
                        if detective_position[i].distance(detective_position[i2]) <= distance_max:
                            delete_index2.append(idx[i2])
                            fl_match.append(str(fl_detective['SOL_standard'].values[i2]))
                            fl_detective = determine_wavelengths(fl_detective, i, i2)
                            #fl_detective = fl_detective.drop((idx[i2]))
                            num_duplicates+=1
                            
                            
        if fl_match == []: 
             fl_match = [0]
        associated_fl.append(fl_match)
        i+=1
    fl_detective.loc[:, 'associated_fl'] = associated_fl
    k = 0
    for elem in associated_fl:
        if elem!=[0]:
            is_associated_fl[k] = 1
        k+=1
    delete_index = set(delete_index)
    delete_index2 = set(delete_index2)
    #print delete_index ==delete_index2
    fl_detective.loc[:, 'is_associated_fl'] = is_associated_fl
    fl_detective = fl_detective.drop(delete_index2) 
    
    print 'original number of events:  %d' % num_events
    print 'duplicated events deleted:  %d' % num_del
    print 'duplicated events merged:  %d' % (len(delete_index2)-num_del)
    print 'new number of events:  %d' % (num_events-len(delete_index2))

    return fl_detective     

In [7]:
fl_set = pd.read_csv('flare_tester.csv', delimiter = ',', header = 0)
fl_detective_clean = fl_duplicates_detective2(fl_set, 30, 60, 10)
fl_detective_clean = add_dist_frm_center_column(fl_detective_clean)
flare_keywords = list(np.genfromtxt('keywords_flare.csv', delimiter=',', dtype=str))
flare_keywords.extend(['is_associated_fl', 'associated_fl', 'is_131', 'is_171', 'is_193', 'is_211', 'is_304', 
                          'is_335','is_94', 'fl_peakflux_131', 'fl_peakflux_171','fl_peakflux_193', 'fl_peakflux_211',
                          'fl_peakflux_304', 'fl_peakflux_335', 'fl_peakflux_94', 'dist_frm_center'])
fl_detective_clean.to_csv('flare_tester_clean.csv', index = False, columns = flare_keywords)



original number of events:  50
duplicated events deleted:  0
duplicated events merged:  18
new number of events:  32


In [ ]:
fl_detective = pd.DataFrame(fl_detective)
fl_detective_clean = fl_duplicates_detective(fl_detective, 20, 60, 10)
fl_detective_clean = add_dist_frm_center_column(fl_detective_clean)
flare_keywords = list(np.genfromtxt('keywords_flare.csv', delimiter=',', dtype=str))
flare_keywords.extend(['is_associated_fl', 'associated_fl', 'is_131', 'is_171', 'is_193', 'is_211', 'is_304', 
                          'is_335','is_94', 'fl_peakflux_131', 'fl_peakflux_171','fl_peakflux_193', 'fl_peakflux_211',
                          'fl_peakflux_304', 'fl_peakflux_335', 'fl_peakflux_94', 'dist_frm_center'])
fl_detective_clean.to_csv('flares_database.csv', index = False, columns = flare_keywords)

In [13]:
# delta_t_max = 10 #[minutes]
# distance_max = 5 #[arcsec, HPC]
# keywords_SSW = ['SOL_standard', 'event_starttime', 'event_endtime', 'event_peaktime', 'hpc_coord', 
#                 'hpc_bbox', 'fl_goescls']

def fl_duplicates_SSW(fl_SSW, delta_t_max, distance_max):
    delete_index = []
    i = 0
    SSW_start = fl_SSW['event_starttime']
    SSW_start = map(parse_time, SSW_start)
    SSW_position = map(lambda x: wkt.loads(x), fl_SSW['hpc_coord'])
    delta_t_max = datetime.timedelta(minutes = delta_t_max)
    
    for elem in SSW_start:
        j = 1
        is_duplicate = True
        while is_duplicate == True and i+j < len(SSW_start):
            is_duplicate = False
            if fl_SSW['fl_goescls'].values[i] == fl_SSW['fl_goescls'].values[i+j]:
                if SSW_start[i+j]-elem <= delta_t_max:
                    if SSW_position[i].distance(SSW_position[i+j]) <= distance_max:
                        delete_index.append(i+j)
                        is_duplicate = True
                        j+=1
        i+=1
    fl_SSW = fl_SSW.drop(delete_index)
    print delete_index
    print fl_SSW.shape
        
    return fl_SSW     

In [23]:
def fl_duplicates_SSW1(fl_SSW):
    delete_index = []
    i = 0
    end = map(parse_time, fl_SSW['event_endtime'])
    
    for i in range(len(fl_SSW['SOL_standard'])-1):
        j = i + 1
        if fl_SSW['SOL_standard'].values[i] == fl_SSW['SOL_standard'].values[j]:
            if end[i] <= end[j]:
                delete_index.append(i)
            else: delete_index.append(j)
    
    fl_SSW = fl_SSW.drop(delete_index)
    
    print len(delete_index)
    
    return fl_SSW
    

In [24]:
fl_SSW = pd.read_csv('raw_SSW.csv', delimiter = ',', header = 0 )
fl_duplicates_SSW = fl_duplicates_SSW1(fl_SSW)
keywords_SSW = list(np.genfromtxt('keywords_SSW_flare.csv', delimiter=',', dtype=str))
fl_duplicates_SSW.to_csv('prepped_SSW.csv', index = False, columns = keywords_SSW)

/usr/local/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (115) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


642


In [ ]:
delta_t_max = 10
distance_max = 10
fl_SSW = fl_duplicates_SSW(fl_SSW, delta_t_max, distance_max)

In [ ]:
#match flares from flare_detective with GOES class
#locations/times don't precisely match up between SSW entries and flare_detective entries...even though they are being 
#collected from the same satellite...

def fl_goes_class():

In [ ]:
def create_fl_database(start_time, end_time, delta_t_max, distance_max):
    fl_detective = client.query(hek.attrs.Time(start_time.isoformat(),end_time.isoformat()), 
                                hek.attrs.EventType('FL'), hek.attrs.FRM.Name == 'Flare Detective - Trigger Module')
    fl_detective = pd.DataFrame(fl_detective)
#     fl_SSW= client.query(hek.attrs.Time(start_time.isoformat(),end_time.isoformat()), hek.attrs.EventType('FL'), 
#                          hek.attrs.FRM.Name == 'SSW Latest Events')
#     fl_SSW = pd.Dataframe(fl_SSW)
    delta_t_max = datetime.timedelta(minutes = delta_t_max)
    fl_detective = fl_duplicates_detective(delta_t_max, distance_max)
#     fl_SSW = fl_duplicates_SSW(delta_t_max, distance_max)
    
    flare_keywords = list(np.genfromtxt('keywords_flare.csv', delimiter=',', dtype=str))
    flare_keywords.extend(['is_associated_fl', 'associated_fl', 'is_131', 'is_171', 'is_193', 'is_211', 'is_304', 
                          'is_335','is_94', 'fl_peakflux_131', 'fl_peakflux_171','fl_peakflux_193', 'fl_peakflux_211',
                          'fl_peakflux_304', 'fl_peakflux_335', 'fl_peakflux_94', 'fl_goescls'])
    fl_detective.to_csv('flares_database.csv', index = False, columns = flare_keywords)

    
    

In [4]:
def find_dist_frm_center(x, y):
    r = 966
    z_sq = (r**2 - x**2 - y**2)
    if z_sq >= 0: z = z_sq**(0.5)
    else: z = (-z_sq)**(0.5)
    dist = (x**2 + y**2 + (z-r)**2)**(0.5)
    return dist

In [5]:
def add_dist_frm_center_column(df):
    dist_frm_center = []
    for i in range(df.shape[0]):
        dist = find_dist_frm_center(df['hpc_x'].values[i], df['hpc_y'].values[i])
        dist_frm_center.append(dist)
    df.loc[:, 'dist_frm_center'] = dist_frm_center
    return df
    

In [ ]:
#TEST

fl_detective = client.query(hek.attrs.Time(start_time.isoformat(),end_time.isoformat()), 
                                hek.attrs.EventType('FL'), hek.attrs.FRM.Name == 'Flare Detective - Trigger Module')

In [4]:
def collect_ar_noaa(df, folder):
    unique_noaa = df.ar_noaanum.unique()
    keywords_ar = list(np.genfromtxt('keywords_ar.csv', delimiter=',', dtype =str))
    noaa_dic = {elem : pd.DataFrame for elem in unique_noaa}
    start_times = []
    end_times= []
    file_names = []
    i = 0
    for key in noaa_dic.keys():
        if i != 0:
            noaa_dic[key] = df[:][df.ar_noaanum == key]
            start_time = noaa_dic[key]['event_starttime'].values[0]
            end_time = noaa_dic[key]['event_endtime'].values[-1]
            start_times.append(start_time)
            end_times.append(end_time)
            fName = str(key)[:-2] +'_'+ start_time[0:10]+'_'+end_time[0:10]
            location = folder + '/' + fName+ '.csv'
            noaa_dic[key].to_csv(location, columns = keywords_ar, index = False)
            file_names.append(fName)
        i+=1  
#     file_names = np.array(file_names).T
#     with open(folder+'/file_names.csv','w') as f:
#         np.savetxt(f, X =file_names, fmt = '%s', delimiter=',')
 

#, hek.attrs.AR.NOAANum != None

In [3]:
ar_keywords = np.genfromtxt('keywords_ar_1.csv', delimiter = ',', dtype = str)

def merge_ar_sources(inputFile, ar_keywords):
    ar_set =pd.read_csv(inputFile, delimiter = ',', header = 0)
    ar_set_NOAA = ar_set.ix[ar_set['frm_identifier']=='NOAA SWPC']
    listOindices = ar_set_NOAA.index.tolist()
    keywords_ar_add = ['ar_mcintoshcls', 'ar_mtwilsoncls', 'ar_numspots']
    ar_set.sort_values(by=['event_starttime','frm_identifier'], ascending =[False, True])
    if listOindices!= [] and listOindices!=range(0, ar_set.shape[0]):
        for i in range(0, listOindices[0]):
            for elem in keywords_ar_add:
                ar_set.loc[i, elem] = ar_set.loc[listOindices[0], elem] 
        for i in range(listOindices[-1], ar_set.shape[0]):
            for elem in keywords_ar_add:
                ar_set.loc[i, elem] = ar_set.loc[listOindices[-1], elem]
        for i, idx in enumerate(listOindices):
            if idx!=listOindices[-1]:
                for j in range(idx, listOindices[i+1]):
                    for elem in keywords_ar_add:
                        ar_set.loc[j, elem] = ar_set.loc[idx, elem]
        ar_set = ar_set.drop(listOindices)
        ar_set.to_csv(inputFile, index = False, columns = ar_keywords)
        
    return ar_set

#merge_ar_sources('11940_2014-01-01_2014-01-04.csv', ar_keywords)
        

In [4]:
import os

ar_keywords = np.genfromtxt('keywords_ar_1.csv', delimiter = ',', dtype = str)

for root, dirs, files in os.walk('AR1'):
    for f in files:
        if f[0] != '.':
            fname = 'AR1/'+f
            _ = merge_ar_sources(fname, ar_keywords)